In [1]:
import pandas as pd

<h1>UsersRecommend</h1>

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [2]:
# Cargamos los archivos CSV a utilizar:
games_df = pd.read_csv(r"..\Archivos Producidos\games.csv")
reviews_df = pd.read_csv(r"..\Archivos Producidos\reviews.csv")

# Convertimos algunos tipos de columna que no se cargaron correctamente:
reviews_df["item_id"] = reviews_df["item_id"].astype("Int64")
reviews_df["year"] = reviews_df["year"].astype("Int64")

In [3]:
games_df.dtypes

app_name                     object
release_year                  int64
item_id                       int64
developer                    object
Action                        int64
Adventure                     int64
Animation &amp; Modeling      int64
Audio Production              int64
Casual                        int64
Design &amp; Illustration     int64
Early Access                  int64
Education                     int64
Free to Play                  int64
Indie                         int64
Massively Multiplayer         int64
Photo Editing                 int64
RPG                           int64
Racing                        int64
Simulation                    int64
Software Training             int64
Sports                        int64
Strategy                      int64
Utilities                     int64
Video Production              int64
Web Publishing                int64
dtype: object

In [4]:
reviews_df.dtypes

user_id               object
item_id                Int64
recommend               bool
review                object
year                   Int64
sentiment_analysis     int64
dtype: object

Ya cargados los archivos, comenzamos con las transformaciones:

In [5]:
# Unimos las columnas que nos serán de utilidad:
df_r_g = pd.merge(games_df[["app_name", "item_id"]], reviews_df[["item_id", "recommend", "year", "sentiment_analysis"]], on="item_id")
df_r_g.head()

,app_name,item_id,recommend,year,sentiment_analysis
0,Carmageddon max pack,282010,True,<NA>,1
1,Half-life,70,True,2015,1
2,Half-life,70,True,2011,1
3,Half-life,70,True,2014,1
4,Half-life,70,True,2013,2


In [6]:
print(len(df_r_g)) # Chequeamos la cantidad de registros

48832


In [7]:
# Eliminamos registros donde "recommend" sea False:
df_r_g = df_r_g[df_r_g["recommend"]]

# Eliminamos registros donde "sentiment_analysis" sea 0:
df_r_g = df_r_g[df_r_g["sentiment_analysis"] != 0]

# Eliminamos registros donde "year" sea nulo:
df_r_g = df_r_g[df_r_g["year"] != None]

In [8]:
print(len(df_r_g)) # Chequeamos la cantidad de registros

32812


In [9]:
# Eliminamos la columnas que ya no utilizaremos:
df_r_g.drop(columns=["item_id", "recommend", "sentiment_analysis"], inplace=True)
df_r_g.head()

,app_name,year
1,Half-life,2015
2,Half-life,2011
3,Half-life,2014
4,Half-life,2013
5,Half-life,2013


In [10]:
# Agregamos una columna de recomendaciones:
df_r_g["recommendation"] = 1

# Agrupamos por "app_name" y "year" y contamos las recomendaciones:
df_usersrecommend = df_r_g.groupby(["app_name", "year"]).agg({"recommendation": "count"}).reset_index()

# Lo ordenamos en cantidad de recomendaciones descendiente:
df_usersrecommend = df_usersrecommend.sort_values(by="recommendation", ascending=False)

df_usersrecommend.head()

,app_name,year,recommendation
2867,Team fortress 2,2014,1430
626,Counter-strike: global offensive,2015,1352
625,Counter-strike: global offensive,2014,955
2866,Team fortress 2,2013,738
1244,Garry's mod,2014,680


Desarrollamos la función:

In [11]:
# Creamos la función UsersRecommend:
def UsersRecommend(año):

    # Corroboramos que el año introducido sea un número:
    if isinstance(año, int):
        
        # En caso de que el año no se encuentre en el DataFrame:
        if año not in df_usersrecommend["year"].tolist(): return f"{año} no se encuentra entre los años disponibles."

        # Caso contrario (ideal):
        else:

            # Filtramos por año:
            df_año = df_usersrecommend[df_usersrecommend["year"] == año].head(3)

            # Creamos la variable que utilizaremos como respuesta final:
            respuesta = [{"Puesto 1": None}, {"Puesto 2": None}, {"Puesto 3": None}]

            # Rellenamos esa variable:
            respuesta[0]["Puesto 1"] = df_año.iloc[0]["app_name"]
            respuesta[1]["Puesto 2"] = df_año.iloc[1]["app_name"]
            respuesta[2]["Puesto 3"] = df_año.iloc[2]["app_name"]

            # Devolvemos la respuesta final:
            return respuesta


    # En caso de que el año introducido no sea un número:
    else: return f"'{año}' no es un año válido."

In [12]:
# Probamos la función
print(UsersRecommend("Prueba"))
print(UsersRecommend( 2015))
print(UsersRecommend(  2018 ))
print(UsersRecommend(2018))
print(UsersRecommend(2010))
print(UsersRecommend(2013 ))

'Prueba' no es un año válido.
[{'Puesto 1': 'Counter-strike: global offensive'}, {'Puesto 2': 'Team fortress 2'}, {'Puesto 3': "Garry's mod"}]
2018 no se encuentra entre los años disponibles.
2018 no se encuentra entre los años disponibles.
[{'Puesto 1': 'Team fortress 2'}, {'Puesto 2': 'Killing floor'}, {'Puesto 3': 'Alien swarm'}]
[{'Puesto 1': 'Team fortress 2'}, {'Puesto 2': "Garry's mod"}, {'Puesto 3': 'Left 4 dead 2'}]


Guardamos el DataFrame utilizado:

In [13]:
# Guardamos el dataframe "df_usersrecommend":
df_usersrecommend.to_csv(r"..\Archivos Producidos\Data para Endpoints\df_usersrecommend.csv", index=False)